# Q3

## Generate Training and Test Data

In [1]:
import os
import random
from PIL import Image, ImageDraw

# Function to ensure the directory for storing data exists
def ensure_data_dir_exists(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"Directory created: {dir_path}")
    else:
        print(f"Directory already exists: {dir_path}")

# Function to draw a star shape
def draw_star(draw, center, size):
    x, y = center
    points = [
        (x, y - size),  # Top
        (x + size * 0.5, y - size * 0.3),
        (x + size, y),
        (x + size * 0.5, y + size * 0.3),
        (x, y + size),
        (x - size * 0.5, y + size * 0.3),
        (x - size, y),
        (x - size * 0.5, y - size * 0.3)
    ]
    draw.polygon(points, fill=0)

# Function to create images of macro-objects from micro-objects
def create_macro_image(shape, layout, image_size=(100, 100), save_path="output_image.png"):
    img = Image.new('L', image_size, 255)  # White background image
    draw = ImageDraw.Draw(img)

    for coords in layout:
        if shape == 'circle':
            draw.ellipse([coords, (coords[0] + 5, coords[1] + 5)], fill=0)  # Create circle objects
        elif shape == 'square':
            draw.rectangle([coords, (coords[0] + 5, coords[1] + 5)], fill=0)  # Create square objects
        elif shape == 'triangle':
            draw.polygon([coords, (coords[0] + 5, coords[1]), (coords[0] + 2.5, coords[1] - 5)], fill=0)  # Triangle
        elif shape == 'star':
            draw_star(draw, coords, size=5)  # Draw a star

    img.save(save_path)
    print(f"Image saved at: {save_path}")

# Generate macro-object images with predefined patterns
def generate_macro_images(output_dir, image_count=100):
    ensure_data_dir_exists(output_dir)

    # Define macro-object patterns for various shapes
    macro_shapes = {
        'I': [(40, 10), (40, 20), (40, 30), (40, 40), (40, 50)],  # Pattern "I"
        'L': [(10, 10), (10, 20), (10, 30), (10, 40), (10, 50), (20, 50), (30, 50)],  # Pattern "L"
        'T': [(30, 10), (30, 20), (30, 30), (30, 40), (20, 10), (40, 10)],  # Pattern "T"
        'X': [(20, 20), (30, 30), (40, 40), (40, 20), (20, 40)],  # Pattern "X"
        'U': [(10, 10), (10, 30), (10, 40), (30, 40), (50, 40), (50, 30), (50, 10)],  # Pattern "U"
        'Triangle': [(25, 10), (10, 40), (40, 40)],  # Triangle shape
        'Star': [(25, 20), (15, 40), (35, 40)],  # Star shape layout
        'Square': [(20, 20), (20, 40), (40, 20), (40, 40)],  # Square shape layout
    }

    # Assign random labels for each generated macro-object
    labels = list(macro_shapes.keys())
    shapes = ['circle', 'square', 'triangle', 'star']  # New shape options
    for i in range(image_count):
        label = random.choice(labels)
        pattern = macro_shapes[label]
        shape = random.choice(shapes)  # Choose a random shape for each pattern

        # Generate the macro-object from micro-object shapes
        output_image = os.path.join(output_dir, f"{label}_{i}.png")
        create_macro_image(shape, pattern, save_path=output_image)

# Run the macro-object generator for training data
image_output_dir = './generated_images_v2'  # Updated training data directory
generate_macro_images(image_output_dir, image_count=100)

# Run the macro-object generator for test data
test_output_dir = './generated_images_test_v2'  # Updated test data directory
generate_macro_images(test_output_dir, image_count=50)  # Generate fewer images for testing


Directory already exists: ./generated_images_v2
Image saved at: ./generated_images_v2/X_0.png
Image saved at: ./generated_images_v2/Triangle_1.png
Image saved at: ./generated_images_v2/U_2.png
Image saved at: ./generated_images_v2/I_3.png
Image saved at: ./generated_images_v2/Triangle_4.png
Image saved at: ./generated_images_v2/Square_5.png
Image saved at: ./generated_images_v2/T_6.png
Image saved at: ./generated_images_v2/Star_7.png
Image saved at: ./generated_images_v2/T_8.png
Image saved at: ./generated_images_v2/X_9.png
Image saved at: ./generated_images_v2/Star_10.png
Image saved at: ./generated_images_v2/X_11.png
Image saved at: ./generated_images_v2/Star_12.png
Image saved at: ./generated_images_v2/X_13.png
Image saved at: ./generated_images_v2/Triangle_14.png
Image saved at: ./generated_images_v2/Star_15.png
Image saved at: ./generated_images_v2/Star_16.png
Image saved at: ./generated_images_v2/Triangle_17.png
Image saved at: ./generated_images_v2/U_18.png
Image saved at: ./gen

## Define Dataset Class and CNN Model

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

# Define the custom dataset class to load images and labels
class MacroObjectDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.image_files = os.listdir(folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_file = os.path.join(self.folder, self.image_files[index])
        image = Image.open(image_file).convert('L')  # Convert image to grayscale
        label = self.image_files[index].split('_')[0]
        label_dict = {"I": 0, "L": 1, "T": 2, "X": 3, "U": 4, "Triangle": 5, "Star": 6, "Square": 7}  # Added 'Square'
        label_id = label_dict[label]

        if self.transform:
            image = self.transform(image)

        return image, label_id

# Define the CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=8):  # Change from 7 to 8
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, num_classes)  # Updated num_classes to 8

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 25 * 25)  # Flatten tensor for fully connected layer
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x


## Load Dataset and Define Training Parameters

In [3]:
# Setup for device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=8).to(device)  # Updated to 8 classes

# Define the transformation for images
image_transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])

# Load the training dataset and create the DataLoader
train_dataset = MacroObjectDataset('./generated_images_v2', transform=image_transform)  # Updated training directory
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Model, loss, and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Train the Model, Print Epoch-wise Accuracy Table & Evaluate the Model on the Test Dataset

In [4]:
# Dictionary to store accuracies for each epoch
epoch_accuracies = {}

# Function to train the CNN model
def train_model(model, data_loader, loss_fn, optimizer, epochs=10):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct_preds = 0
        total_samples = 0

        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate running accuracy
            running_loss += loss.item()
            _, predictions = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_preds += (predictions == labels).sum().item()

        accuracy = 100 * correct_preds / total_samples
        epoch_accuracies[epoch + 1] = accuracy  # Store accuracy for the epoch
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(data_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Train the model and print accuracy table
train_model(model, train_loader, loss_function, optimizer)

# Print results as a table
print("\nEpoch vs Accuracy:")
print("{:<10} {:<15}".format("Epoch", "Accuracy (%)"))
for epoch, accuracy in epoch_accuracies.items():
    print("{:<10} {:<15.2f}".format(epoch, accuracy))

# Testing Function
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Load the test dataset and create the DataLoader
test_dataset = MacroObjectDataset('./generated_images_test_v2', transform=image_transform)  # Updated test directory
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Test the model after training
test_model(model, test_loader)


Epoch [1/10], Loss: 1.7754, Accuracy: 44.77%
Epoch [2/10], Loss: 0.2617, Accuracy: 97.05%
Epoch [3/10], Loss: 0.0159, Accuracy: 100.00%
Epoch [4/10], Loss: 0.0024, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0010, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0005, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0004, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0003, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0003, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0002, Accuracy: 100.00%

Epoch vs Accuracy:
Epoch      Accuracy (%)   
1          44.77          
2          97.05          
3          100.00         
4          100.00         
5          100.00         
6          100.00         
7          100.00         
8          100.00         
9          100.00         
10         100.00         
Test Accuracy: 100.00%


# Q4

## Generate Dataset Using the Principle of Continuity

In [7]:
import os
import random
import math  # Import math for trigonometric functions
from PIL import Image, ImageDraw

# Create a folder to save generated images
def create_data_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory created: {path}")
    else:
        print(f"Directory already exists: {path}")

# Function to generate continuity-based shapes
def generate_continuity_shapes(image_dir, num_images=100):
    create_data_folder(image_dir)

    for i in range(num_images):
        shape_type = random.choice(['Curve_Line', 'Straight_Line', 'Zigzag_Line'])

        if shape_type == 'Curve_Line':
            arrangement = [(x, int(50 + 20 * math.sin(x / 10))) for x in range(10, 90, 5)]  # Curve pattern
        elif shape_type == 'Straight_Line':
            arrangement = [(x, 50) for x in range(10, 90, 5)]  # Straight horizontal line
        else:
            arrangement = [(x, 50 + (x % 2) * 10) for x in range(10, 90, 5)]  # Zigzag line

        output_path = os.path.join(image_dir, f"{shape_type}_{i}.png")
        create_continuity_object_image(arrangement, shape_type, output_path=output_path)

# Function to create continuity-based images
def create_continuity_object_image(arrangement, shape_type, image_size=(100, 100), output_path="output.png"):
    img = Image.new('L', image_size, color=255)  # Create a white background image
    draw = ImageDraw.Draw(img)

    for idx in range(len(arrangement) - 1):
        draw.line([arrangement[idx], arrangement[idx + 1]], fill=0, width=5)

    img.save(output_path)
    print(f"Saved image: {output_path}")

# Generate the continuity dataset
continuity_dataset_dir = './data_continuity'
generate_continuity_shapes(continuity_dataset_dir, num_images=100)


Directory already exists: ./data_continuity
Saved image: ./data_continuity/Curve_Line_0.png
Saved image: ./data_continuity/Zigzag_Line_1.png
Saved image: ./data_continuity/Zigzag_Line_2.png
Saved image: ./data_continuity/Zigzag_Line_3.png
Saved image: ./data_continuity/Straight_Line_4.png
Saved image: ./data_continuity/Zigzag_Line_5.png
Saved image: ./data_continuity/Curve_Line_6.png
Saved image: ./data_continuity/Zigzag_Line_7.png
Saved image: ./data_continuity/Zigzag_Line_8.png
Saved image: ./data_continuity/Curve_Line_9.png
Saved image: ./data_continuity/Curve_Line_10.png
Saved image: ./data_continuity/Straight_Line_11.png
Saved image: ./data_continuity/Curve_Line_12.png
Saved image: ./data_continuity/Curve_Line_13.png
Saved image: ./data_continuity/Curve_Line_14.png
Saved image: ./data_continuity/Curve_Line_15.png
Saved image: ./data_continuity/Straight_Line_16.png
Saved image: ./data_continuity/Straight_Line_17.png
Saved image: ./data_continuity/Zigzag_Line_18.png
Saved image: ./d

## Define the Dataset Class and Transformations

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Custom Dataset class to load continuity-based images and labels
class ContinuityShapesDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.image_names = os.listdir(image_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_names[idx])
        image = Image.open(img_name).convert('L')  # Convert to grayscale

        label_name = self.image_names[idx].split('_')[0]  # e.g., 'Curve' from 'Curve_Line_1'
        label_map = {'Curve': 0, 'Straight': 1, 'Zigzag': 2}
        label = label_map[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])


## Define the CNN Model

In [9]:
import torch.nn as nn
import torch.optim as optim

# Define the CNN Model
class CNN(nn.Module):
    def __init__(self, num_classes=3):  # We have 3 classes: Curve, Straight, Zigzag
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 1 channel (grayscale images)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 25 * 25, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = x.view(-1, 64 * 25 * 25)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Setup for device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(num_classes=3).to(device)


## Train the Model on the Generated Dataset

In [10]:
# Load the training dataset
train_dataset = ContinuityShapesDataset(image_folder=continuity_dataset_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Model, loss, and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs)


Epoch [1/10], Loss: 1.2565, Accuracy: 49.37%
Epoch [2/10], Loss: 0.5644, Accuracy: 79.11%
Epoch [3/10], Loss: 0.1355, Accuracy: 100.00%
Epoch [4/10], Loss: 0.0139, Accuracy: 100.00%
Epoch [5/10], Loss: 0.0008, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0001, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0000, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0000, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0000, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0000, Accuracy: 100.00%


## Evaluate the Model on a Separate Test Set

In [11]:
# Generate test dataset
test_dataset_dir = './data_continuity_test'
generate_continuity_shapes(test_dataset_dir, num_images=20)

# Load the test dataset
test_dataset = ContinuityShapesDataset(image_folder=test_dataset_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Evaluate the model on the test dataset
evaluate_model(model, test_loader)


Directory already exists: ./data_continuity_test
Saved image: ./data_continuity_test/Curve_Line_0.png
Saved image: ./data_continuity_test/Curve_Line_1.png
Saved image: ./data_continuity_test/Curve_Line_2.png
Saved image: ./data_continuity_test/Zigzag_Line_3.png
Saved image: ./data_continuity_test/Straight_Line_4.png
Saved image: ./data_continuity_test/Zigzag_Line_5.png
Saved image: ./data_continuity_test/Straight_Line_6.png
Saved image: ./data_continuity_test/Zigzag_Line_7.png
Saved image: ./data_continuity_test/Zigzag_Line_8.png
Saved image: ./data_continuity_test/Straight_Line_9.png
Saved image: ./data_continuity_test/Straight_Line_10.png
Saved image: ./data_continuity_test/Straight_Line_11.png
Saved image: ./data_continuity_test/Curve_Line_12.png
Saved image: ./data_continuity_test/Curve_Line_13.png
Saved image: ./data_continuity_test/Curve_Line_14.png
Saved image: ./data_continuity_test/Zigzag_Line_15.png
Saved image: ./data_continuity_test/Zigzag_Line_16.png
Saved image: ./data_co

100.0